# Example of use of submission tools  

**The following is a brief example of how to import and use `submit.py` (or any script within the tools folder).**  

For this we will recreate (partially) the baseline submission, but with much cleaner code.  

### Note - relative import
If we try to use a relative import path in our notebook

```python
from ..tools import submit
```
we get a "relative import error". This is mainly due to how imports work in iPython notebooks. Easiest way around it is to temporarily add the path to the `scripts/` subfolder to our `python path` (the set of locations in which python will look to import packages). See below:


In [1]:
# For relative imports
import sys
sys.path.append('/Users/will/Github/facial-keypoint-detection/scripts')

# Import submit module from our tools subfolder
from tools import submit

Now we are good to use any function in our `submit.py` module. Since we commented our module appropriately, we can even pull up the "documentation":

In [2]:
submit.create_submission?

## Baseline submission

Now that we have our package loaded let's recreate the baseline model for the first feature.  

But first let's do our normal imports

In [3]:
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsRegressor

In [ ]:
### TODO - tools.getdata

Now let us load the data: (**TODO - create a tool/getdata.py module**)  

In [4]:
# Load data
full_data = pd.read_csv("../../data/datasets/training.csv")

# Preprocess images into unsigned 8byte integer arrays
get_img = lambda x: np.uint8( map(int, x.split()) )
full_data['img_processed'] = map( get_img, full_data['Image'])

# fill na
data_nonas = full_data.dropna()

# TRAINING DATA
train_data = data_nonas.iloc[:,31].values
train_labels = np.round(data_nonas.iloc[:,:30].values)

# For base submission no (dev data)

# TEST DATA
full_test = pd.read_csv("../../data/datasets/test.csv")

# Preprocess images into unsigned 8byte integer arrays
full_test['img_processed'] = map( get_img, full_test['Image'])

# Get training data
test_data = full_test[['img_processed']].copy()

Create and train a KNN model on the first feature

In [5]:
# KNN model
knn = KNeighborsRegressor()
knn.fit(train_data.tolist(), train_labels[:,0])

models =  [ (data_nonas.columns[0], knn) ]

And now we can enjoy the `tools.submit` module!

In [6]:
submit.create_generate(test_data, models, 'tools_example', verbose=True)

Predicting "left_eye_center_x"... done! (55.5s)
... Created the csv file: ../../data/submissions/tools_example_submission.csv


In [7]:
%ls ../../data/submissions/

IdLookupTable.csv                       full_knregressor_submission_nonull.csv
README.md                               tools_example_submission.csv
SampleSubmission.csv


This function is just a wrapper, we could've used the functions `submit.create_submission` and then `submit.generate_csv`:

```python
# Create predictions
kn_predictions = submit.create_submission(test_data, models, 'tools_example')

# Generate submition csv from predictions
submit.generate_csv(kn_predictions, 'tools_example')
```